In [26]:
from keras.models import Sequential
from keras import layers
from keras.layers import LeakyReLU
from keras import optimizers
from keras.datasets import mnist
import numpy as np
from PIL import Image
import argparse
import math
from keras.layers.core import Activation

In [43]:
def my_generator():
    model = Sequential()
    model.add(layers.Dense(input_dim=100, output_dim=512))
    model.add(Activation('relu'))
    model.add(layers.Dense(128*49))
    model.add(layers.BatchNormalization())
    model.add(Activation('relu'))
    model.add(layers.Reshape((7, 7, 128), input_shape=(128*49,)))
    model.add(layers.UpSampling2D(size=(2, 2)))
    model.add(layers.Conv2D(64, (5, 5), padding='same'))
    model.add(Activation('relu'))
    model.add(layers.UpSampling2D(size=(2, 2)))
    model.add(layers.Conv2D(1, (3, 3), padding='same'))
    model.add(Activation('tanh'))
    return model

In [44]:
def my_discriminator():
    model = Sequential()
    model.add(layers.Conv2D(32,(5,5),padding='same',input_shape=(28,28,1))) #mnist image dimensions (28*28*1)
    model.add(LeakyReLU(alpha=0.2))
    model.add(layers.MaxPooling2D(pool_size=(2,2)))
    model.add(layers.Conv2D(64,(5,5)))
    model.add(layers.BatchNormalization())
    model.add(LeakyReLU(alpha=0.2))
    model.add(layers.Conv2D(128,(3,3)))
    model.add(layers.BatchNormalization())
    model.add(LeakyReLU(alpha=0.2))
    model.add(layers.MaxPooling2D(pool_size=(2,2)))
    model.add(layers.Flatten())
    model.add(layers.Dense(512))
    model.add(layers.BatchNormalization())
    model.add(LeakyReLU(alpha=0.2))
    model.add(layers.Dense(1))
    model.add(LeakyReLU(alpha=0.2))
    return model
    
    
    
    
    

In [45]:
def images_combined(generated_images):
    num = generated_images.shape[0]
    width = int(math.sqrt(num))
    height = int(math.ceil(float(num)/width))
    shape = generated_images.shape[1:3]
    image = np.zeros((height*shape[0], width*shape[1]),
                     dtype=generated_images.dtype)
    for index, img in enumerate(generated_images):
        i = int(index/width)
        j = index % width
        image[i*shape[0]:(i+1)*shape[0], j*shape[1]:(j+1)*shape[1]] = \
            img[:, :, 0]
    return image

In [46]:
##### Fake model for loss function ########
def my_loss_model(generator, discriminator):
    model = Sequential()
    model.add(generator)
    discriminator.trainable= False # the loss of discriminator will be passed on to the generator
    model.add(discriminator)
    return model

In [ ]:
################################### Training ################################
BATCH_SIZE=100
(X_train, y_train), (X_test, y_test) = mnist.load_data()
X_train = (X_train.astype(np.float32) - 127.5)/127.5
X_train = X_train[:, :, :, None]
X_test = X_test[:, :, :, None]

discriminator = my_discriminator()
generator = my_generator()
loss_3 = my_loss_model(generator,discriminator)
disc_op = optimizers.SGD(lr=0.0005, momentum=0.9, nesterov=True)
gen_op = optimizers.SGD(lr=0.0005, momentum=0.9, nesterov=True)
generator.compile(loss='binary_crossentropy', optimizer="SGD")
loss_3.compile(loss='binary_crossentropy', optimizer=gen_op)
discriminator.trainable = True
discriminator.compile(loss='binary_crossentropy', optimizer=disc_op)

for epoch in range(100):
    print("Epoch is", epoch)
    print("Number of batches", int(X_train.shape[0]/BATCH_SIZE))
    for index in range(int(X_train.shape[0]/BATCH_SIZE)):
        noise = np.random.uniform(-1, 1, size=(BATCH_SIZE, 100))
        image_batch = X_train[index*BATCH_SIZE:(index+1)*BATCH_SIZE]
        generated_images = generator.predict(noise, verbose=0)
        if index % 20 == 0:
            image = images_combined(generated_images)
            image = image*127.5+127.5
            Image.fromarray(image.astype(np.uint8)).save(
                str(epoch)+"_"+str(index)+".png")
        X = np.concatenate((image_batch, generated_images))
        y = [1] * BATCH_SIZE + [0] * BATCH_SIZE
        d_loss = discriminator.train_on_batch(X, y)
        print("batch %d d_loss : %f" % (index, d_loss))
        noise = np.random.uniform(-1, 1, (BATCH_SIZE, 100))
        discriminator.trainable = False
        g_loss = loss_3.train_on_batch(noise, [1] * BATCH_SIZE)
        discriminator.trainable = True
        print("batch %d My mixed loss is : %f" % (index, g_loss))
        if index % 10 == 9:
            generator.save_weights('generator', True)
            discriminator.save_weights('discriminator', True)


In [ ]:
def generate(BATCH_SIZE, nice=False):
    generator = my_generator()
    generator.compile(loss='binary_crossentropy', optimizer="SGD")
    generator.load_weights('generator')
    if nice:
        discriminator = my_discriminator()
        discriminator.compile(loss='binary_crossentropy', optimizer="SGD")
        discriminator.load_weights('discriminator')
        noise = np.random.uniform(-1, 1, (BATCH_SIZE*20, 100))
        generated_images = generator.predict(noise, verbose=1)
        d_pret = discriminator.predict(generated_images, verbose=1)
        index = np.arange(0, BATCH_SIZE*20)
        index.resize((BATCH_SIZE*20, 1))
        pre_with_index = list(np.append(d_pret, index, axis=1))
        pre_with_index.sort(key=lambda x: x[0], reverse=True)
        nice_images = np.zeros((BATCH_SIZE,) + generated_images.shape[1:3], dtype=np.float32)
        nice_images = nice_images[:, :, :, None]
        for i in range(BATCH_SIZE):
            idx = int(pre_with_index[i][1])
            nice_images[i, :, :, 0] = generated_images[idx, :, :, 0]
        image = combine_images(nice_images)
    else:
        noise = np.random.uniform(-1, 1, (BATCH_SIZE, 100))
        generated_images = generator.predict(noise, verbose=1)
        image = combine_images(generated_images)
    image = image*127.5+127.5
    Image.fromarray(image.astype(np.uint8)).save(
        "generated_image.png")